# search 

Lizette Carpenter 

This journal runs examples for the following functions: 

1. **find_complete_list**: Finds all smaller diagonals (and the associated pairs of repeats) that are contained pair_list, which is composed of larger diagonals found in find_initial_repeats. 

2. **find_add_srows**: Finds pairs of repeated structures, represented as diagonals of a certain length, k, that start at the same time step as previously found pairs of repeated structures of the same length.

3. **find_add_erows**: Finds pairs of repeated structures, represented as diagonals of a certain length, k, that end at the same time step as previously found pairs of repeated structures of the same length.

4. **find_add_mrows** - Finds pairs of repeated structures, represented as diagonals of a certain length, k, that neither start nor end at the same time steps as previously found pairs of repeated structures of the same length.   

5. **find_all_repeats**: Finds all the diagonals present in thresh_mat. This function is nearly identical to find_initial_repeats, with two crucial differences. First, we do not remove diagonals after we find them. Second, there is no smallest bandwidth size as we are looking for all diagonals.

6. **find_complete_list_anno_only**: Finds annotations for all pairs of repeats found in find_all_repeats. This list contains all the pairs of repeated structures with their start/end indices and lengths.    

Imported functions used from utilities include:  

* stretch_diags
* add_annotations 
* \_\_find_song_pattern

<h3 align = "left"> Figure 1. Where are you? </h3>
<img src="function_flow_chart.png" alt="Chart" style="width:250px;" align = "left"/>

- Figure 1 is a function flowchart of aligned hierarchies. This flowchart shows each function that is called when example.py is executed. This journal focuses on the functions highlighted in purple which together make up the search module. Few of the functions highlighted in yellow such as add_annotations.py and stretch_diags.py are called by certain search functions. Such functions belong to the utilities module. Hierarchical_structure.py is an assemble function which calls one of the search functions. Assemble functions are highlighted in red. In addition, hierarchical_structure calls remove_overlaps, highlighted in green. These functions are included in the transform module.

## Import Modules

In [ ]:
import numpy as np
import search 
from search import *
from utilities import stretch_diags
from utilities import add_annotations
from utilities import __find_song_pattern

-------
## 1. find_complete_list

### About find_complete_list 

As seen in the flow chart, `find_intial_repeats` is called by `example` right before `find_complete_list`. In `find_complete_list`, smaller pairs of repeats are added to the original list of pairs of repeats made in `find_initial_repeats`. All of the pairs of repeats correspond to each repeated structure in another numpy array called thresh_mat. This array holds all the repeated structures in a sequential data stream and the repeated structures are represented as diagonals.   

The following search functions get called in `find_complete_list`: 
   * find_add_erows
        * from utilities import
            - add_annotions 
            - \_\_find_song_pattern
   * find_add_srows
       * from utilities import
            - add_annotions 
            - \_\_find_song_pattern
   * find_add_mrows
        * from utilities import
            - add_annotions 
            - \_\_find_song_pattern

-------
### Arguments:

- pair_list : numpy array 
   - list of pairs of repeats found in `find_initial_repeats.py`
   - each row represents a pair of a repeated structure found in a sequential data stream 
       - the first and and second column represnts the start and end indices  of a repeated structure 
       - the third and fourth column represents the start and end indices of the other repeated structure
       - the fifth column represents the bandwidth of the repeated structure

- song_length : integer 
    - the number of audio shingles or the length of the song 

--------
### Returns: 

- lst_out : numpy array 
    - list of pairs of repeats with smaller repeats added  
--------
### MatLab¶

Example 1:

`ans =   
           1  10  46  55  10  1 
           1  40  46  55  10  1 
           1  15  31  45  15  1 
          10  20  40  50  15  2 `
          
------

In [ ]:
# Example 1
# Inputs: 
pair_list = np.array([[1, 15, 31, 45, 15], 
                      [1, 10, 46, 55, 10], 
                      [31, 40, 46, 55, 10],
                      [10, 20, 40, 50, 15]])
song_length = 55

In [ ]:
find_complete_list(pair_list, song_length)

------
When running `find_complete_list.py`, you may have noticed an IndexError on line 786 in `search.py` where add_annotations.py is called to find the annotations markers for the current pairs of repeated structures in the for loop and output as a numpy array called temp_anno_lst. 

Within `add_annotations.py`, `__find_song_pattern.py` is called which raises the original index error. 

To still see what `find_complete_list.py` outputs, we will import find_complete_list1 from search_mod. Here, temp_anno_lst is hard-coded with the output it would have had if `add_annotations.py` had worked. Outputs comes from Matlab*. 

---
*Procedure of obtaining `temp_anno_lst` from `find_complete_list.m` in Matlab:
- Download all of the files needed to run find_complete_lst.m from [kmkinnaird/ThesisCode](https://github.com/kmkinnaird/ThesisCode/tree/master/MATLABcode). 
    - find_complete_list.m 
    - find_add_srows_both_check_no_anno.m
    - find_add_erows_both_check_no_anno.m
    - find_add_mrows_both_check_no_anno.m
    - add_annotations.m 
    - stitch_diags.m (this is equivalent to `__find_song_pattern.py`) 
- In Matlab
    - run each file 
    - open find_complete_list 
    - under line 100, where add_annotations.m is called 
        `[temp_anno_lst] = add_annotations(temp_anno_lst, sn);`
      add the following two lines, 
         `fprintf('temp_anno_lst');`
         `disp(temp_anno_lst);`
      and then save. 
    - in the console, initialize pair_lst and sn 
    - in the console, run find_complete_list(pair_lst, sn) to get results 
----

In [ ]:
import numpy as np
from search_mod import find_complete_list1

In [ ]:
# Scroll down to see how temp_anno_lst is hard-coded. You will find it before the final for-loop, commented as Step 3.
# Under Part C of Step 3, you will see the call to add_annotations commented out. 
%cat find_complete_list1.py

In [ ]:
# find_complete_list1
# Inputs: 
pair_list = np.array([[1, 15, 31, 45, 15], 
                      [1, 10, 46, 55, 10], 
                      [31, 40, 46, 55, 10],
                      [10, 20, 40, 50, 15]])
song_length = 55

In [ ]:
find_complete_list1(pair_list, song_length)

-----------------------------------------
## 2. find_add_srows

### About find_add_srows
Finds pairs of repeated structures, representated as diagonals of a certain length that start at the same time step as previously found pairs of repeated structures of the same length. 
* Called by **find_complete_list.py**
* from utilities import
    - add_annotions 
    - \_\_find_song_pattern
---
### Arguments

- lst_no_anno: numpy array 
    - list of pairs of repeats
        
- check_inds: numpy array
    - list of ending indices for repeats of length k that we 
        use to check lst_no_anno for more repeats of length k 
       
- k: int
    - length of repeats that we are looking for
            
---
### Returns: 

- add_rows: np.array
    - List of newly found pairs of repeats of length K that are contained in larger repeats in lst_no_anno
---

### MatLab Output

`ans = 
     1    10    31    40    10
    11    15    41    45     5
     1    10    31    40    10
    11    15    41    45     5`
    
---

In [ ]:
%cat find_add_srows.py

In [ ]:
# find_add_srows:
# Example 1 Inputs: 
lst_no_anno = np.array([[ 1, 15, 31, 45, 15],
                        [ 1, 10, 46, 55, 10],
                        [31, 40, 46, 55, 10],
                        [10, 20, 40, 50, 15]])
check_inds = np.array([ 1, 31, 46])
k = 10

In [ ]:
find_add_srows(lst_no_anno, check_inds, k)

---
## 3. find_add_erows

### About find_add_erows
Finds pairs of repeated structures, representated as diagonals of a 
certain length that end at the same time step as previously found pairs of repeated structures of the same length.

* Called by **find_complete_list.py**
* from utilities import 
    - add_annotions 
    - \_\_find_song_pattern
    
---
### Arguments

- lst_no_anno: numpy array
    - list of pairs of repeats
        
- check_inds: numpy array
    - list of ending indices for repeats of length k that we use 
      to check lst_anno_no for more repeats of length k
        
- k: int
    - length of repeats that we are looking for 

---
### Returns: 
- add_rows: numpy array
    - list of newly found pairs of repeats of length k that are contained in larger repeats in lst_no_anno
---

### Matlab Output 

**Example 1:** 

`ans = 
     []`
    
    
---

In [ ]:
# find_add_erows 
# Example 1 Inputs: 
lst_no_anno = np.array([[ 1, 15, 31, 45, 15],
                        [ 1, 10, 46, 55, 10],
                        [31, 40, 46, 55, 10],
                        [10, 20, 40, 50, 15]])
check_inds = np.array([10, 40, 55])
k = 10

In [ ]:
find_add_erows(lst_no_anno, check_inds, k)

---
## 4. find_add_mrows

### About find_add_mrows
Finds pairs of repeated structures, represented as diagonals of a certain
length that neither start nor end at the same time steps as previously
found pairs of repeated structures of the same length. 
* Called by **find_complete_list.py**
* from utilities import 
    - add_annotions 
    - \_\_find_song_pattern
    
---
### Arguments

- lst_no_anno: numpy array 
    - list of pairs of repeats

- check_inds: numpy array 
    - list of ending indices for repeats of length k that we use to check lst_no_anno for more repeats of length k 

- k: integer
    - length of repeats that we are looking for 

---
### Returns: 

- add_rows: numpy array 
    - list of newly found pairs of repeats of length K that are contained in larger repeats in lst_no_anno
---

### Matlab Output 

**Example 1:** 

`ans = 
     []`
    
    
---

In [ ]:
# find_add_mrows: 
# Inputs: 
lst_no_anno = np.array([[ 1, 15, 31, 45, 15],
                        [ 1, 10, 46, 55, 10],
                        [31, 40, 46, 55, 10],
                        [10, 20, 40, 50, 15]])
check_inds = np.array([ 1, 31, 46])
k = 10

In [ ]:
find_add_mrows(lst_no_anno, check_inds, k)

---
## 5. find_all_repeats 
**this function currently does not work.**
### About find_all_repeats 

Finds all the diagonals present in thresh_mat. This function is nearly identical to find_initial_repeats, with two crucial differences. First, we do not remove diagonals after we find them. Second, there is no smallest bandwidth size as we are looking for all diagonals.
* Called by **hierarchical_structure.py**  
* Uses one utility function
    - from utilities import 
        - stretch_diags
---    
### Arguments 

- thresh_mat: numpy array
    - thresholded matrix that we extract diagonals from
    
- band_width_vec: numpy array
    - vector of lengths of diagonals to be found
---    
### Returns 

- all_lst: numpy array
    - list of pairs of repeats that correspond to diagonals in thresh_mat
        
---
### MatLab Output 

Note: Matlab function is called `lightup_lst_with_thresh_bw_no_remove.m`

*Example 1:* 
    
`ans =
    2 2 4 4 1`
    
---

In [ ]:
# find_all_repeats 
# Inputs: 
thresh_mat = np.array([[0, 0, 0, 0, 0],
                       [0, 1, 0, 1, 0],
                       [0, 0, 0, 0, 0],
                       [0, 1, 0, 1, 0],
                       [0, 0, 0, 0, 0]])

bandwidth_vec = np.array([[1, 2, 3, 4, 5]])

In [ ]:
find_all_repeats(thresh_mat, bandwidth_vec)

---
## 6. find_complete_list_anno_only

### About find_complete_list_anno_only 

Finds annotations for all pairs of repeats found in `find_all_repeats.py`. This list contains all the pairs of repeated structures with their start/end indices and lengths.

* Called by **hierarchical_structure.py**
* Uses two utilities functions
    - from utilities import
        - add_annotions 
        - \_\_find_song_pattern
---    
### Arguments 

- pair_list: numpy array 
    - list of pairs of repeats
    - WARNING: bandwidths must be in ascending order
        
- song_length: int
    - number of audio shingles in song

---
### Returns 

- out_lst: numpy array 
    - list of pairs of repeats with smaller repeats added and with annotation markers
--- 

### MatLab Output 
` ans = 
     2 2 4 4 1 1`
 

In [ ]:
# find_complete_list_anno_only 
# Inputs: 
pair_list = np.array([[2, 2, 4, 4, 1]])
song_length = 5 

In [ ]:
find_complete_list_anno_only(pair_list, song_length)